# qBraid-SDK: Transpiler

In [7]:
import numpy as np

from qbraid.programs import load_program, QPROGRAM_REGISTRY
from qbraid.interface import (
    circuits_allclose,
    assert_allclose_up_to_global_phase,
    random_circuit,
)
from qbraid.transpiler import transpile

The qBraid transpiler supports all-to-all connectivity between the following quantum program types:

In [2]:
QPROGRAM_REGISTRY

{'cirq': cirq.circuits.circuit.Circuit,
 'qiskit': qiskit.circuit.quantumcircuit.QuantumCircuit,
 'pennylane': pennylane.tape.tape.QuantumTape,
 'pyquil': pyquil.quil.Program,
 'pytket': pytket._tket.circuit.Circuit,
 'braket': braket.circuits.circuit.Circuit,
 'openqasm3': openqasm3.ast.Program,
 'qasm2': str,
 'qasm3': str}

## Basic usage example: Qiskit $\leftrightarrow$ Amazon Braket $\leftrightarrow$ Cirq

In [3]:
from qiskit import QuantumCircuit


def test_circuit():
    circuit = QuantumCircuit(4)

    circuit.h([0, 1, 2, 3])
    circuit.x([0, 1])
    circuit.y(2)
    circuit.z(3)
    circuit.s(0)
    circuit.sdg(1)
    circuit.t(2)
    circuit.tdg(3)
    circuit.rx(np.pi / 4, 0)
    circuit.ry(np.pi / 2, 1)
    circuit.rz(3 * np.pi / 4, 2)
    circuit.p(np.pi / 8, 3)
    circuit.sx(0)
    circuit.sxdg(1)
    circuit.iswap(2, 3)
    circuit.swap([0, 1], [2, 3])
    circuit.cx(0, 1)
    circuit.cp(np.pi / 4, 2, 3)

    return circuit

We'll start with a 4-qubit qiskit circuit that uses 15 unique gates

In [4]:
qiskit_circuit = test_circuit()
qiskit_circuit.draw()

┌───┐┌───┐ ┌───┐ ┌─────────┐   ┌────┐                 
q_0: ┤ H ├┤ X ├─┤ S ├─┤ Rx(π/4) ├───┤ √X ├───X────────■────
     ├───┤├───┤┌┴───┴┐├─────────┤  ┌┴────┴┐  │      ┌─┴─┐  
q_1: ┤ H ├┤ X ├┤ Sdg ├┤ Ry(π/2) ├──┤ √Xdg ├──┼──X───┤ X ├──
     ├───┤├───┤└┬───┬┘├─────────┴┐┌┴──────┴┐ │  │   └───┘  
q_2: ┤ H ├┤ Y ├─┤ T ├─┤ Rz(3π/4) ├┤0       ├─X──┼──■───────
     ├───┤├───┤┌┴───┴┐└┬────────┬┘│  Iswap │    │  │P(π/4) 
q_3: ┤ H ├┤ Z ├┤ Tdg ├─┤ P(π/8) ├─┤1       ├────X──■───────
     └───┘└───┘└─────┘ └────────┘ └────────┘

In [8]:
braket_circuit = transpile(qiskit_circuit, "braket")
print(braket_circuit)

WARNING - This program uses OpenQASM language features that may not be supported on QPUs or on-demand simulators.


T  : │  0  │  1  │  2   │       3       │  4  │  5  │  6  │  7  │   8    │   9    │      10       │
      ┌───┐ ┌───┐ ┌───┐   ┌──────────┐   ┌───┐                                                     
q0 : ─┤ H ├─┤ X ├─┤ S ├───┤ Rx(0.79) ├───┤ V ├──────────────────────x─────────────────────●────────
      └───┘ └───┘ └───┘   └──────────┘   └───┘                      │                     │        
      ┌───┐ ┌───┐ ┌────┐  ┌──────────┐   ┌───┐ ┌───┐ ┌───┐          │                   ┌─┴─┐      
q1 : ─┤ H ├─┤ X ├─┤ Si ├──┤ Ry(1.57) ├───┤ S ├─┤ H ├─┤ S ├──────────┼────────x──────────┤ X ├──────
      └───┘ └───┘ └────┘  └──────────┘   └───┘ └───┘ └───┘          │        │          └───┘      
      ┌───┐ ┌───┐ ┌───┐   ┌──────────┐   ┌───┐ ┌───┐       ┌───┐    │        │                     
q2 : ─┤ H ├─┤ Y ├─┤ T ├───┤ Rz(2.36) ├───┤ S ├─┤ H ├───●───┤ X ├────x────────┼────────────●────────
      └───┘ └───┘ └───┘   └──────────┘   └───┘ └───┘   │   └─┬─┘             │            │        


In [9]:
cirq_circuit = transpile(qiskit_circuit, "cirq")
print(cirq_circuit)

q_0: ───H───X───S──────Rx(0.25π)───X^0.5────────────────×───────@────────
                                                        │       │
q_1: ───H───X───S^-1───Ry(0.5π)────X^-0.5───────────────┼───×───X────────
                                                        │   │
q_2: ───H───Y───T──────Rz(0.75π)───S────────H───@───X───×───┼───@────────
                                                │   │       │   │
q_3: ───H───Z───T^-1───Z^(1/8)─────S────────────X───@───H───×───@^0.25───


Qubit indexing varies between packages, so some circuit diagrams appear flipped, but the matrix representations are equivalent.

To verify, we'll use the sdk's `circuits_allclose` function, which applies the agnostic `qbraid.interface.to_unitary` function to each of two input circuits, checks the matricies against `np.allclose`, and returns the result.

In [10]:
circuits_allclose(qiskit_circuit, braket_circuit) and circuits_allclose(
    braket_circuit, cirq_circuit
)

True

## Stress-testing against randomly generated circuits

For a second example, we'll generate some even larger circuits, and do so randomly, to test the limits of the transpiler.

The qBraid-SDK has its own `random_circuit` function that takes in any supported package as an argument, but to show that there's no pre-processing or filtering going on behind the scenes, we'll use functions from cirq's testing module to generate circuits and to check equivalance after transpiling.

In [11]:
import cirq

kwargs = {
    "num_qubits": np.random.randint(8, 11),
    "depth": np.random.randint(8, 11),
    "op_density": np.random.randint(80, 100) / 100,
    "random_state": np.random.randint(1, 11),
}

circuit_start = random_circuit("cirq", **kwargs)
start_u = circuit_start.unitary()
print("num_qubits:", len(circuit_start.all_qubits()))
print("depth:", len(circuit_start))
print("op_density:", kwargs["op_density"])
print(f"matrix dimension: {start_u.shape}\n")
print(circuit_start)

num_qubits: 10
depth: 8
op_density: 0.97
matrix dimension: (1024, 1024)

      ┌───────────┐   ┌──────┐   ┌───────┐   ┌───┐   ┌───────┐   ┌───────┐           ┌──────┐
0: ────S───────────────iSwap──────X────────────@──────Z───────────iSwap──────Z────────Y─────────
                       │          │            │                  │
1: ──────────iSwap─────┼────×─────@───────────Y┼────────────×─────┼────@─────iSwap─────────@────
             │         │    │                  │            │     │    │     │             │
2: ─────────@┼─────────iSwap┼─────T───────────X┼──────@─────┼─────iSwap┼─────┼────────Y────┼────
            ││              │                 ││      │     │          │     │             │
3: ────@────┼┼─────────Y────┼─────────────────┼┼S─────@─────┼──────────@─────iSwap────iSwap┼────
       │    ││              │                 ││            │                         │    │
4: ────X────┼┼─────────Y────┼──────iSwap──────┼┼───────iSwap┼───────────@────Y────────iSwap┼────
 

Starting with this randomly generated circuit, we'll repeatedly apply the qbraid circuit wrapper and transpile from one supported package to the next until we arrive all the way back at a cirq circuit.

In [12]:
braket_circuit = transpile(circuit_start, "braket")
print(type(braket_circuit))

<class 'braket.circuits.circuit.Circuit'>


In [13]:
pyquil_circuit = transpile(braket_circuit, "pyquil")
print(type(pyquil_circuit))

<class 'pyquil.quil.Program'>


In [14]:
qiskit_circuit = transpile(pyquil_circuit, "qiskit")
print(type(qiskit_circuit))

<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>


In [15]:
pytket_circuit = transpile(qiskit_circuit, "pytket")
print(type(pytket_circuit))

<class 'pytket._tket.circuit.Circuit'>


In [16]:
circuit_finish = transpile(pytket_circuit, "cirq")
print(type(circuit_finish))

<class 'cirq.circuits.circuit.Circuit'>


Computing the final unitary and checking its shape

In [17]:
finish_u = circuit_finish.unitary()
print(finish_u.shape)

(1024, 1024)


In [18]:
try:
    assert_allclose_up_to_global_phase(start_u, finish_u, atol=1e-7)
    print("Test passed!")
except AssertionError:
    print("Test failed")

Test passed!
